# **Voice Gender Classification using AI**
#### **Author:** Emmanuel Oludare Ejifolabi
#### **Date:** July 2024

In [50]:
import os
import shutil
import joblib
import numpy as np
import pandas as pd
import librosa
import kagglehub
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

## CONFIGURATION

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
SAMPLE_RATE = 22050
N_MFCC = 13
DATASET_ID = "/content/drive/MyDrive/DSP_PROJECTS/Gender_Voice_Classification/Gender_Dataset/data"
# BASE_DIR = "dataset/"
MODEL_PATH = "/content/drive/MyDrive/DSP_PROJECTS/Gender_Voice_Classification/voice_gender_model.pkl"

## FEATURE EXTRACTION

In [64]:
def extract_features(file_path):
    try:
        y, sr = librosa.load(file_path, sr=SAMPLE_RATE)

        mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=N_MFCC).T, axis=0)
        chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr).T, axis=0)
        centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr).T, axis=0)
        zcr = np.mean(librosa.feature.zero_crossing_rate(y=y).T, axis=0)

        return np.hstack([mfcc, chroma, centroid, zcr])
    except Exception as e:
        print(f"Error extracting features from {file_path}: {e}")
        return None

## LOAD DATASET

In [65]:
def load_dataset(data_dir):
    X, y = [], []
    for gender in ['male', 'female']:
        gender_dir = os.path.join(data_dir, gender)
        if not os.path.isdir(gender_dir):
            continue

        for filename in os.listdir(gender_dir):
            if filename.lower().endswith(".wav"):
                file_path = os.path.join(gender_dir, filename)
                features = extract_features(file_path)
                if features is not None:
                    X.append(features)
                    y.append(gender)
    return np.array(X), np.array(y)

## TRAIN THE MODEL

In [66]:
def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    print("\n=== 📊 Evaluation Results ===")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
    joblib.dump(model, MODEL_PATH)
    print(f"✅ Model saved to: {MODEL_PATH}")
    return model


## PREDICT NEW AUDIO

In [67]:
def predict(model, file_path):
    features = extract_features(file_path)
    if features is not None:
        prediction = model.predict([features])[0]
        print(f"Predicted gender: {prediction}")
        return prediction
    else:
        print("Could not extract features.")
        return None


## MAIN EXECUTION

In [68]:
if __name__ == "__main__":
    print("📥 Importing dataset from drive")
    dataset_path = DATASET_ID
    print(f"✅ Dataset downloaded at: {dataset_path}")

    print("📚 Loading data and extracting features...")
    X, y = load_dataset(dataset_path)

    print("🤖 Training the model...")
    model = train_model(X, y)

📥 Importing dataset from drive
✅ Dataset downloaded at: /content/drive/MyDrive/DSP_PROJECTS/Gender_Voice_Classification/Gender_Dataset/data
📚 Loading data and extracting features...
🤖 Training the model...

=== 📊 Evaluation Results ===
Confusion Matrix:
[[1135   19]
 [  17 2059]]

Classification Report:
              precision    recall  f1-score   support

      female       0.99      0.98      0.98      1154
        male       0.99      0.99      0.99      2076

    accuracy                           0.99      3230
   macro avg       0.99      0.99      0.99      3230
weighted avg       0.99      0.99      0.99      3230

✅ Model saved to: /content/drive/MyDrive/DSP_PROJECTS/Gender_Voice_Classification/voice_gender_model.pkl


## Predict a sample

In [69]:
    predict(model, os.path.join(dataset_path, "/content/drive/MyDrive/DSP_PROJECTS/Gender_Voice_Classification/cleaned_airport_spectral_gate.wav"))

Predicted gender: female


np.str_('female')